In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### 1. Data Exploration and Preprocessing: 
Understand your data. Handle missing values, encode categorical variables, and normalize numerical variables.

In [3]:
df = pd.read_csv("data/train.csv")

print(df.shape)
df.head()

(1460, 81)


Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD         Normal     208500  
1   2007        WD         Normal     181500  
2   2008        WD         Normal     223500  
3   2006        WD        Abnorml     140000  
4   2008        WD         Normal     250000  

[5 rows x 81 columns]

In [4]:
#check null values and print only which have null values
print(df.isnull().sum()[df.isnull().sum() > 0])

#print % of missing values in each column
print(df.isnull().mean()[df.isnull().sum() > 0] * 100)



LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64
LotFrontage     17.739726
Alley           93.767123
MasVnrType      59.726027
MasVnrArea       0.547945
BsmtQual         2.534247
BsmtCond         2.534247
BsmtExposure     2.602740
BsmtFinType1     2.534247
BsmtFinType2     2.602740
Electrical       0.068493
FireplaceQu     47.260274
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
GarageQual       5.547945
GarageCond       5.547945
PoolQC          99.520548
Fence           80.753425
MiscFeature     96.301370
dtype: float64


In [5]:
# dropping cols with missing values > 40%
cols_to_drop = df.columns[df.isnull().mean() > 0.4]
df = df.drop(cols_to_drop, axis=1)
#print % of missing values in each column
print(df.isnull().mean()[df.isnull().sum() > 0] * 100)

LotFrontage     17.739726
MasVnrArea       0.547945
BsmtQual         2.534247
BsmtCond         2.534247
BsmtExposure     2.602740
BsmtFinType1     2.534247
BsmtFinType2     2.602740
Electrical       0.068493
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
GarageQual       5.547945
GarageCond       5.547945
dtype: float64


In [6]:
cols_to_impute = df.columns[df.isnull().mean() > 0]

#check col types
print(df[cols_to_impute].dtypes)

#impute float cols with mean and object cols with mode
for col in cols_to_impute:
    if df[col].dtype == 'float64':
        df[col] = df[col].fillna(df[col].mean())
    else:
        df[col] = df[col].fillna(df[col].mode()[0])

print(df.isnull().mean()[df.isnull().sum() > 0] * 100)



LotFrontage     float64
MasVnrArea      float64
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Electrical       object
GarageType       object
GarageYrBlt     float64
GarageFinish     object
GarageQual       object
GarageCond       object
dtype: object
Series([], dtype: float64)


In [7]:
#print all col names
print(df.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'Enclo

In [8]:
# #count plots
# for col in df.columns:
#     if df[col].dtype == 'object':
#         plt.figure()
#         df[col].value_counts().plot.bar()
#         plt.title(col)
#         plt.show()


You will create a new variable 'House Category' based on 'House Style', 'Bldg Type', 'Year Built', and 'Year Remod/Add' for the classification task.

In [9]:
cols_for_category = ["HouseStyle","BldgType","YearBuilt","YearRemodAdd"]

#data types of cols
print(df[cols_for_category].dtypes)

#display unique values in each col
for col in cols_for_category:
    print(col)
    print(df[col].unique())



HouseStyle      object
BldgType        object
YearBuilt        int64
YearRemodAdd     int64
dtype: object
HouseStyle
['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
BldgType
['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
YearBuilt
[2003 1976 2001 1915 2000 1993 2004 1973 1931 1939 1965 2005 1962 2006
 1960 1929 1970 1967 1958 1930 2002 1968 2007 1951 1957 1927 1920 1966
 1959 1994 1954 1953 1955 1983 1975 1997 1934 1963 1981 1964 1999 1972
 1921 1945 1982 1998 1956 1948 1910 1995 1991 2009 1950 1961 1977 1985
 1979 1885 1919 1990 1969 1935 1988 1971 1952 1936 1923 1924 1984 1926
 1940 1941 1987 1986 2008 1908 1892 1916 1932 1918 1912 1947 1925 1900
 1980 1989 1992 1949 1880 1928 1978 1922 1996 2010 1946 1913 1937 1942
 1938 1974 1893 1914 1906 1890 1898 1904 1882 1875 1911 1917 1872 1905]
YearRemodAdd
[2003 1976 2002 1970 2000 1995 2005 1973 1950 1965 2006 1962 2007 1960
 2001 1967 2004 2008 1997 1959 1990 1955 1983 1980 1966 1963 1987 1964
 1972 1996 1998 1989 195

## Creating Categories from 
- House Style
- Bldg Type
- Year Built
- Year Remod/Add

In [10]:
# Define category mapping
def categorize(row):
    if row['YearBuilt'] >= 1990 or row['YearRemodAdd'] >= 1990:
        era = 'Modern'
    elif row['YearBuilt'] < 1950 and row['YearRemodAdd'] < 1975:
        era = 'Historic'
    else:
        era = 'Mid-Century'

    if 'Twnhs' in row['BldgType']:
        type_ = 'Townhouse'
    elif row['BldgType'] in ['1Fam', '2fmCon']:
        type_ = 'Family Home'
    else:
        type_ = 'Multi-Unit'

    if row['HouseStyle'] in ['1Story', '1.5Fin', '2Story']:
        style_ = '1Story'
    elif row['HouseStyle'] in ['2Story', '2.5Fin', '2.5Unf']:
        style_ = '2Story'
    else:
        style_ = 'Other'

    return f"{era} {type_} {style_}"

# Apply categorization
df['house_category'] = df.apply(categorize, axis=1)


In [11]:
#plot house category
plt.figure()
df['house_category'].value_counts().plot.bar()
plt.title('house_category')
plt.show()



In [12]:
#print all cols
print(df.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'Enclo

In [13]:
#drop ID col
df = df.drop("Id", axis=1)


In [14]:
#print cols datatypes
for col in df.columns:
    print(col, df[col].dtype)

MSSubClass int64
MSZoning object
LotFrontage float64
LotArea int64
Street object
LotShape object
LandContour object
Utilities object
LotConfig object
LandSlope object
Neighborhood object
Condition1 object
Condition2 object
BldgType object
HouseStyle object
OverallQual int64
OverallCond int64
YearBuilt int64
YearRemodAdd int64
RoofStyle object
RoofMatl object
Exterior1st object
Exterior2nd object
MasVnrArea float64
ExterQual object
ExterCond object
Foundation object
BsmtQual object
BsmtCond object
BsmtExposure object
BsmtFinType1 object
BsmtFinSF1 int64
BsmtFinType2 object
BsmtFinSF2 int64
BsmtUnfSF int64
TotalBsmtSF int64
Heating object
HeatingQC object
CentralAir object
Electrical object
1stFlrSF int64
2ndFlrSF int64
LowQualFinSF int64
GrLivArea int64
BsmtFullBath int64
BsmtHalfBath int64
FullBath int64
HalfBath int64
BedroomAbvGr int64
KitchenAbvGr int64
KitchenQual object
TotRmsAbvGrd int64
Functional object
Fireplaces int64
GarageType object
GarageYrBlt float64
GarageFinish object


Separate the input features (X) from the target variables (Y):


In [15]:
X = df.drop(['SalePrice', 'house_category'], axis=1)
y_price = df['SalePrice']
y_category = df['house_category']

In [16]:
#y_category datatype
print(y_price)



0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64


Perform any necessary data preprocessing, such as handling missing values, encoding categorical variables, and scaling numerical features. You can use techniques like imputation, one-hot encoding, and normalization/standardization.

In [17]:
#integer encoding y_category
y_category = y_category.astype('category')
y_category = y_category.cat.codes

#print head
print(y_category.head(10))

#y_category datatype
print(y_category.dtype)

#plot y_category value counts
plt.figure()
y_category.value_counts().plot.bar()
plt.title("y_category")
plt.show()

0    10
1     4
2    10
3     0
4    10
5    10
6    10
7     4
8     0
9     2
dtype: int8
int8


In [18]:
#one hot encoding for X
X = pd.get_dummies(X, drop_first=True)
#print cols datatypes
print(X.dtypes.value_counts())
print("-----------------\n after conversion\n-----------------")
#convert to float64
for col in X.columns:
    X[col] = X[col].astype('float64')

print(X.dtypes.value_counts())



bool       193
int64       33
float64      3
Name: count, dtype: int64
-----------------
 after conversion
-----------------
float64    229
Name: count, dtype: int64


In [19]:
X.shape

(1460, 229)

Split the data into training and testing sets:


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_price_train, y_price_test, y_category_train, y_category_test = train_test_split(
    X, y_price, y_category, test_size=0.2, random_state=42
)

Further split the training data into training and validation sets:


In [21]:
X_train, X_val, y_price_train, y_price_val, y_category_train, y_category_val = train_test_split(
    X_train, y_price_train, y_category_train, test_size=0.2, random_state=42
)

Create PyTorch datasets and data loaders for training, validation, and testing:

In [22]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Create PyTorch datasets
train_dataset = TensorDataset(torch.tensor(X_train.values, dtype=torch.float32),
                              torch.tensor(y_price_train.values, dtype=torch.float32),
                              torch.tensor(y_category_train.values, dtype=torch.long))

val_dataset = TensorDataset(torch.tensor(X_val.values, dtype=torch.float32),
                            torch.tensor(y_price_val.values, dtype=torch.float32),
                            torch.tensor(y_category_val.values, dtype=torch.long))

test_dataset = TensorDataset(torch.tensor(X_test.values, dtype=torch.float32),
                             torch.tensor(y_price_test.values, dtype=torch.float32),
                             torch.tensor(y_category_test.values, dtype=torch.long))

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



: 

In [ ]:
input_size = X_train.shape[1]
num_classes = len(y_category.unique())
print(input_size, num_classes)

232 17


Define your PyTorch Lightning model with shared layers and task-specific heads for 'SalePrice' and 'category' prediction,

In [ ]:
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

class HousingModel(pl.LightningModule):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super().__init__()
        self.hidden_layer1 = nn.Linear(input_size, hidden_size1)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size1)  # BatchNorm for the first hidden layer
        # self.layer_norm1 = nn.LayerNorm(hidden_size1)  # LayerNorm for the first hidden layer
        self.hidden_layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size2)  # BatchNorm for the second hidden layer
        # self.layer_norm2 = nn.LayerNorm(hidden_size2)  # LayerNorm for the second hidden layer
        self.price_layer = nn.Linear(hidden_size2, 1)  # Regression output
        self.category_layer = nn.Linear(hidden_size2, num_classes)  # Classification output

        self.w_loss = 0.1
        self.w_acc = 1-self.w_loss

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        # Kaiming (He) initialization for layers with ReLU activation
        nn.init.kaiming_normal_(self.hidden_layer1.weight, mode='fan_out', nonlinearity='relu')
        nn.init.kaiming_normal_(self.hidden_layer2.weight, mode='fan_out', nonlinearity='relu')
        # For the output layers, you might use a simpler initialization since they might not be followed by ReLU
        nn.init.xavier_normal_(self.price_layer.weight)
        nn.init.xavier_normal_(self.category_layer.weight)

        
    def forward(self, x):
        x = self.hidden_layer1(x)
        x = self.batch_norm1(x)  # Apply BatchNorm after first linear layer
        # x = self.layer_norm1(x)  # Apply LayerNorm after BatchNorm
        x = torch.relu(x)
        
        x = self.hidden_layer2(x)
        x = self.batch_norm2(x)  # Apply BatchNorm after second linear layer
        # x = self.layer_norm2(x)  # Apply LayerNorm after BatchNorm
        x = torch.relu(x)
        
        price = self.price_layer(x)
        category = self.category_layer(x)
        return price, category
    
    def training_step(self, batch, batch_idx):
        x, y_price, y_category = batch
        price_pred, category_pred = self(x)
        
        price_loss = nn.MSELoss()(price_pred, y_price.view(-1, 1))
        category_loss = nn.CrossEntropyLoss()(category_pred, y_category)
        
        loss = self.w_loss * price_loss + self.w_acc * category_loss
        self.log('train_loss', loss)
        return loss
    

    
    def validation_step(self, batch, batch_idx):
        x, y_price, y_category = batch
        price_pred, category_pred = self(x)
        
        price_loss = nn.MSELoss()(price_pred, y_price.view(-1, 1))
        category_loss = nn.CrossEntropyLoss()(category_pred, y_category)

        loss = self.w_loss * price_loss + self.w_acc * category_loss

        accuracy = accuracy_score(y_category.cpu().numpy(), category_pred.argmax(dim=1).cpu().numpy(), normalize=True)
        precision = precision_score(y_category.cpu().numpy(), category_pred.argmax(dim=1).cpu().numpy(), average='weighted', zero_division=1)
        recall = recall_score(y_category.cpu().numpy(), category_pred.argmax(dim=1).cpu().numpy(), average='weighted', zero_division=1)
        f1 = f1_score(y_category.cpu().numpy(), category_pred.argmax(dim=1).cpu().numpy(), average='weighted', zero_division=1)

   

        self.log('val_loss', loss)
        self.log('val_accuracy', accuracy)
        self.log('val_precision', precision)
        self.log('val_recall', recall)
        self.log('val_f1', f1)
        self.log('val_price_loss', price_loss)
        self.log('val_category_loss', category_loss)
    
    def test_step(self, batch, batch_idx):
        x, y_price, y_category = batch
        price_pred, category_pred = self(x)
        
        price_loss = nn.MSELoss()(price_pred, y_price.view(-1, 1))
        category_loss = nn.CrossEntropyLoss()(category_pred, y_category)

        loss = self.w_loss * price_loss + self.w_acc * category_loss

        accuracy = accuracy_score(y_category.cpu().numpy(), category_pred.argmax(dim=1).cpu().numpy(), normalize=True)
        precision = precision_score(y_category.cpu().numpy(), category_pred.argmax(dim=1).cpu().numpy(), average='weighted', zero_division=1)
        recall = recall_score(y_category.cpu().numpy(), category_pred.argmax(dim=1).cpu().numpy(), average='weighted', zero_division=1)
        f1 = f1_score(y_category.cpu().numpy(), category_pred.argmax(dim=1).cpu().numpy(), average='weighted', zero_division=1)
        
        self.log('test_loss', loss)
        self.log('test_price_loss', price_loss)
        self.log('test_category_loss', category_loss)
        self.log('test_accuracy', accuracy)
        self.log('test_precision', precision)
        self.log('test_recall', recall)
        self.log('test_f1', f1)

    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.1)



In [ ]:
!python -V

Python 3.11.5


In [319]:
# Initialize TensorBoardLogger
logger = TensorBoardLogger("tb_logs", name="my_model")

# Train the model using the Trainer from PyTorch Lightning
input_size = X_train.shape[1]
num_classes = len(y_category.unique())

model = HousingModel(input_size=input_size, hidden_size1=512, hidden_size2=512, num_classes=num_classes)

trainer = pl.Trainer(max_epochs=20, enable_progress_bar=True, logger=logger)
trainer.fit(model, train_loader, val_loader)

#tensorboard --logdir=tb_logs

ModuleNotFoundError: Neither `tensorboard` nor `tensorboardX` is available. Try `pip install`ing either.
DistributionNotFound: The 'tensorboardX' distribution was not found and is required by the application. HINT: Try running `pip install -U 'tensorboardX'`
DistributionNotFound: The 'tensorboard' distribution was not found and is required by the application. HINT: Try running `pip install -U 'tensorboard'`

Evaluate the trained model on the test set:

In [280]:
trainer.validate(model, val_loader)

#RMSE of val_price_loss
rmse = np.sqrt(trainer.logged_metrics['val_price_loss'])
print(f'RMSE Price Val: {rmse:.2f}')

#Accuracy of val_category_loss
accuracy = trainer.logged_metrics['val_accuracy']
print(f'Accuracy: {accuracy*100:.2f}')


/Users/krishan/.pyenv/versions/3.10.13/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.5128205418586731
    val_category_loss         233.31884765625
         val_f1             0.48709389567375183
        val_loss                146000528.0
      val_precision         0.7564496397972107
     val_price_loss            1460003200.0
       val_recall           0.5128205418586731
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
RMSE Price Val: 38209.99
Accuracy: 51.28


In [277]:
trainer.test(model, test_loader)

#RMSE of test_price_loss
rmse = np.sqrt(trainer.logged_metrics['test_price_loss'])
print(f'RMSE Price Test: {rmse:.2f}')

#Accuracy of test_category_loss
accuracy = trainer.logged_metrics['test_accuracy']
print(f'Accuracy: {accuracy*100:.2f}')

/Users/krishan/.pyenv/versions/3.10.13/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          0.232876718044281
   test_category_loss        175.4313507080078
         test_f1            0.19933286309242249
        test_loss               204949648.0
     test_precision         0.6470465660095215
     test_price_loss           2049495168.0
       test_recall           0.232876718044281
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
RMSE Price Test: 45271.35
Accuracy: 23.29
